In [ ]:
import pathlib
from pathlib import Path
from typing import List, Tuple, Union
import copy
import numpy as np
import matplotlib.pyplot as plt
import serpentTools
from serpentTools.parsers.results import ResultsReader
import click
import matplotlib.pyplot as plt

serpentTools.settings.rc["serpentVersion"] = "2.1.32"
P = 48 #max no of FA
Z = 6  #max no of slices


AV = 0.6221408E+24


def discharge_bu() -> None:
    res_list = []

    with serpentTools.settings.rc:
        serpentTools.settings.rc["verbosity"] = "error"
    dep = serpentTools.read("wh_lfr_dep.m")
    
    return dep

dep = discharge_bu()

In [ ]:
materials = dep.materials.keys()

In [ ]:
def split_name(name: str)->Tuple[int, int]:
    without_fuel = name[4:]
    split_without_fuel  = without_fuel.split("Z")
    p_num = int(split_without_fuel[0][1:])
    z_num = int(split_without_fuel[1])
    return p_num, z_num

In [ ]:
materials_dict = {}
for material in materials:
    if material == "total":
        continue
    materials_dict[material] = {"mdens": dep.materials[material].mdens[:, -1].sum(), "burnup": dep.materials[material].burnup[-1]}

In [ ]:
for material in materials_dict:
    p_index, z_index = split_name(str(material))
    mdens_to_divide_key = f"fuelP1Z{z_index}"
    mdens_to_divide = dep.materials[mdens_to_divide_key].mdens[:, -1].sum()
    ratio = materials_dict[material]["mdens"] / mdens_to_divide
    materials_dict[material]["mdens_ratio"] = ratio

In [ ]:
for material in materials_dict.keys():
    p_index, z_index = split_name(material)
    burn_up = materials_dict[material]["burnup"]
    if p_index != 1:
        p_index = p_index - 1
    
    corrected_material = f"fuelP{p_index}Z{z_index}"
    ratio = materials_dict[corrected_material]["mdens_ratio"]
    corrected_burnup = burn_up * ratio
    materials_dict[material]["corrected_burnup"] = corrected_burnup


In [ ]:
average_burnups = {}
for material in materials_dict.keys():
    p_index, _ = split_name(material)
    average_burnup = 0
    for zidx in range(1 , Z + 1):
        key_construct = f"fuelP{p_index}Z{zidx}"
        average_burnup += materials_dict[key_construct]["corrected_burnup"] / Z
    average_burnups[f"P{p_index}"] = average_burnup
    

In [ ]:
sum([value for key, value in average_burnups.items()])

In [ ]:
average_burnups_axial = {}
for material in materials_dict.keys():
    _, z_index = split_name(material)
    average_axial_burnup = 0
    for pidx in range(1 , P + 1):
        key_construct = f"fuelP{pidx}Z{z_index}"
        average_axial_burnup += materials_dict[key_construct]["corrected_burnup"]
    average_burnups_axial[f"Z{z_index}"] = average_axial_burnup
    

In [ ]:
average_burnups_axial

In [ ]:
sum([value for key, value in average_burnups_axial.items()]) / Z

In [1]:
# OLD VERSION

In [ ]:
dep.materials['fuelP48Z1'].mden

In [ ]:
#print(dep.materials["fuelP1Z1"].mdens[:,3][:38])
#print(len(dep.materials["fuelP1Z1"].mdens[:,3][:38]))
print(dep.names[:39])
m_dens = dep.materials["fuelP1Z1"].mdens[:,3][:39]
names = np.array(dep.names[:39])
print(names)
print(m_dens)
print(np.vstack((names,m_dens)))




In [ ]:

last_bu_step = len(dep.burnup) - 1
max_list_HM = np.where(np.char.find(dep.names, "Cm250") == 0)[0][0] + 1

# Gets the correction ration for FA
c_ratio = []
name_list = []
serpent_bu = []
corrected_bu = []

step = 0

for poz in range(1, P+1):
    for slice in range(1, Z+1):
        fa_name = f"fuelP{poz}Z{slice}"
        fa1_name = f"fuelP1Z{slice}"

        sum_hm = np.sum(dep.materials[fa_name].mdens[: ,last_bu_step][:max_list_HM])
        sum_hm_fa1 = np.sum(dep.materials[fa1_name].mdens[: ,last_bu_step][:max_list_HM])
        ratio_i = sum_hm/sum_hm_fa1

        c_ratio.append(ratio_i)
        name_list.append(fa_name)

        
        serpent_bu.append(dep.materials[fa_name].burnup[last_bu_step])

        #print(step)
        if step <= 5:
            #print("FA1")
            corrected_bu.append(dep.materials[fa1_name].burnup[last_bu_step])
        else:    
            #print("FAn")
            fa_c_bu = dep.materials[fa_name].burnup[last_bu_step] * c_ratio[step-6]
            #print(c_ratio[step-6])
            corrected_bu.append(fa_c_bu)

        

        step += 1

#np.vstack((name_list,c_ratio))
#print(len(corrected_bu))
#print(corrected_bu)



In [ ]:
type(serpent_bu)


In [ ]:
c_ratio